In [2]:
# Install packages
# Put all 'pip install' commands here..

!pip install torch
!pip install transformers
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

device = "cpu"
# Move the model to the GPU
if torch.cuda.is_available():
    device = "cuda"

model.to(device)

print(f"device: {device}")

device: cuda


In [54]:
import numpy as np

question = "What is the capital of France?"
context = "Paris is the capital of France."

inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors='pt')
input_ids = inputs["input_ids"].tolist()[0]
attention_mask = inputs["attention_mask"].tolist()[0]

input_ids = torch.tensor([input_ids]).to(device)
attention_mask = torch.tensor([attention_mask]).to(device)

output = model(input_ids, attention_mask=attention_mask)

answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)

if answer_end >= answer_start:
    answer_tokens = input_ids[0][answer_start:answer_end+1].tolist()
    answer_tokens = tokenizer.convert_ids_to_tokens(answer_tokens)
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")

print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


tensor([[ 101, 2054, 2003, 1996, 3007, 1997, 2605, 1029,  102, 3000, 2003, 1996,
         3007, 1997, 2605, 1012,  102]], device='cuda:0')

Question:
What is the capital of france?

Answer:
Paris.
